In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
import pyLDAvis.gensim_models
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models.phrases import Phrases, Phraser
from gensim.models import LdaMulticore
from gensim.models import LdaModel
from gensim.models import CoherenceModel
import yake
from langdetect import detect, detect_langs, DetectorFactory
import re

In [2]:
# read csv
authors = pd.read_csv('top_20_authors.csv')
publications = pd.read_csv('publications-top_20_authors.csv', sep=',')

In [57]:
# make a dictionary containing the combined abstracts for each author
author_publication_pairs = list(zip(publications['user_id'], publications['abstract_text']))

DetectorFactory.seed = 0

authors_texts = {author_id: "" for author_id in authors["id"]}
for author_id, abstract in author_publication_pairs:
    if abstract and isinstance(abstract, str) and re.match('^(?=.*[a-zA-Z])', abstract):
        language = detect(abstract)
        if language != 'en':
            print(language)
        else:
            authors_texts[author_id] = authors_texts[author_id] + "\n" + abstract

ro
ro
ro
it
it
ro
ro
id
id
ro
de
ro
ro
ro
ro
ro
ro
ro
ro
pt
ro
ro
ro
ro
ro
ro
fr
fr
ro
ro
ro
ro
ro
ro
ro
ro
fr
fr
fr
ro
ro
ro
fr
ro
ro
ro
ro
fr
ro
fr
fr
ro
ro
ro
ro
fr
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
fr
ro
ro
de
de
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
fr
ro
ro
ja
ro
ro
fr
es
ro
ro
ro
ro
de
ro
ro
ro
ro
ro
ro
fr
ro
et
ro
ro
ro
et
ro
fr
ro
fr
fr
fr
fr
de
ro
ro
ro
fr
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
pt
pt
ro
ro
fr
et
ro
ro
ro
tr
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
de
de
ro
ro
de
de
de
de
de
de
ro
de
ro
ro
ro
ca
ro
ro
ro
ro
fr
es
fr
ro
de
de
so
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
fr
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
fr
fr
fr
fr
fr
de
de
de
de
de
fr
ro
ro
ro
de
fr
ro
ro
ro
ro
ca
ro
ro
ro
ro
ro
ro
ro
fr
fr
ro
ro
fr
ro
ro
ro
ro
it
ro
fr
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
ro
de
fr


In [58]:
nlp = spacy.load('en_core_web_md')

EXTRACT KEYWORDS/KEYPHRASES WITH YAKE

In [78]:
kw_extractor = yake.KeywordExtractor()
max_ngram = 3
deduplication_threshold = 0.5
keywords_nr = 15
custom_kw_extractor = yake.KeywordExtractor(lan="en", n=max_ngram, dedupLim=deduplication_threshold, top=keywords_nr, features=None)

text = authors_texts[534]
doc = nlp(text)
remove_entities = ['PERSON', 'NORP', 'FAC', 'GPE', 'LOC', 'DATE', 'TIME', 'PERCENT', 'MONEY',
                   'QUANTITY', 'CARDINAL', 'ORDINAL']
stop_words = ['paper', 'present', 'propose', 'show', 'datum', 'people', 'result', 'solution', 'case', 'order',
                  'base', 'ieee', 'privacy', 'policy', 'new', 'old', 'context', 'high', 'different', 'research', 'type',
                  'approach', 'important', 'main', 'range', 'helpful', 'large', 'difficult', 'available', 'amount',
                  'useful', 'importance', 'article', 'abstract', 'scale', 'copyright', 'real', 'quality', 'demonstrate',
                  'inconvenience', 'benefit', 'unavailable', 'term', 'condition', 'interest', 'recent', 'obtain',
                  'organization', 'task', 'student', 'professor', 'teacher', 'university', 'workshop', 'study', 'text', 'jat', 'jats']

transformed_text = ' '.join([token.text for token in doc if token.ent_type_ not in remove_entities
                             and token.lemma_.lower() not in stop_words])

keywords = custom_kw_extractor.extract_keywords(transformed_text)
for kw in keywords:
    print(kw)

('distributed systems', 3.509302360162876e-05)
('Mobile Cloud computing', 3.809210395500882e-05)
('cloud computing systems', 5.410482182189355e-05)
('mobile devices', 8.652648328096323e-05)
('Opportunistic networks', 9.303329906713266e-05)
('network management services', 0.00015152061105763796)
('mobile social networks', 0.00015433899488251414)
('systems', 0.00015734145430671892)
('network', 0.00019232239228677731)
('mobile opportunistic cloud', 0.00020049293754866143)
('Services', 0.00023050105690822903)
('mobile', 0.00023288249418062875)
('intelligent transportation systems', 0.0002460307080771078)
('service computing architectures', 0.0002676603830539024)
('Big Data', 0.00026862447805840105)


EXTRACT KEYWORDS/KEYPHRASES WITH YAKE - from each abstract

In [79]:
# make a dictionary containing a list of abstracts for each author
author_publication_pairs = list(zip(publications['user_id'], publications['abstract_text']))

authors_texts = {author_id:[] for author_id in authors["id"]}
for author_id, abstract in author_publication_pairs:
    if abstract and isinstance(abstract, str) and re.match('^(?=.*[a-zA-Z])', abstract):
        language = detect(abstract)
        if language == 'en':
            authors_texts[author_id].append(abstract)

In [82]:
# concatenate chunks of 25 abstracts
chunk_size = 100
abstracts = authors_texts[534]
new_abstracts = []
abstract_count = len(abstracts)

for i in range(0, abstract_count, chunk_size):
    end = i + chunk_size if i + 2 * chunk_size <= abstract_count else abstract_count
    abstracts_chunk = ' '.join(abstracts[i:i+chunk_size])
    new_abstracts.append(abstracts_chunk)


In [83]:
kw_extractor = yake.KeywordExtractor()
max_ngram = 3
deduplication_threshold = 0.7
keywords_nr = 5
custom_kw_extractor = yake.KeywordExtractor(lan="en", n=max_ngram, dedupLim=deduplication_threshold, top=keywords_nr, features=None)

remove_entities = ['PERSON', 'NORP', 'FAC', 'GPE', 'LOC', 'DATE', 'TIME', 'PERCENT', 'MONEY',
                   'QUANTITY', 'CARDINAL', 'ORDINAL']
stop_words = ['paper', 'present', 'propose', 'show', 'datum', 'people', 'result', 'solution', 'case', 'order',
                  'base', 'ieee', 'privacy', 'policy', 'new', 'old', 'context', 'high', 'different', 'research', 'type',
                  'approach', 'important', 'main', 'range', 'helpful', 'large', 'difficult', 'available', 'amount',
                  'useful', 'importance', 'article', 'abstract', 'scale', 'copyright', 'real', 'quality', 'demonstrate',
                  'inconvenience', 'benefit', 'unavailable', 'term', 'condition', 'interest', 'recent', 'obtain',
                  'organization', 'task', 'student', 'professor', 'teacher', 'university', 'workshop', 'study', 'text', 'jat', 'jats']

for text in new_abstracts:
    doc = nlp(text)
    transformed_text = ' '.join([token.text for token in doc if token.ent_type_ not in remove_entities
                             and token.lemma_.lower() not in stop_words])
    
    keywords = custom_kw_extractor.extract_keywords(transformed_text)
    for kw in keywords:
        print(kw)
    print()

('distributed systems', 0.00014737726273259392)
('Mobile Cloud computing', 0.0001553843845553562)
('CPC Program Library', 0.00019293281743546688)
('distributed system technologies', 0.00021163232168504875)
('Opportunistic Mobile Networks', 0.00022045292001703065)

('distributed systems', 0.00010278648299842698)
('mobile cloud computing', 0.00013423850838873276)
('mobile cloud systems', 0.00015288315547572217)
('Cloud computing systems', 0.00020420207726288753)
('distributed system technologies', 0.0002947597311836742)

('Intelligent Transportation Systems', 0.00031589866062634497)
('opportunistic networks', 0.00035659915944817806)
('network management services', 0.0003662044185913426)
('distributed systems', 0.0004116821210382379)
('Opportunistic network applications', 0.0004933637909109081)

('technical professional dedicated', 5.537009241867369e-05)
('web site signifies', 9.52271798327684e-05)
('advancing technology', 0.0002662045235255443)
('distributed systems', 0.00037107763923394

EXTRACT TOPICS WITH GENSIM LDA

In [120]:
# make a dictionary containing a list of abstracts for each author
author_publication_pairs = list(zip(publications['user_id'], publications['abstract_text']))

authors_texts = {author_id:[] for author_id in authors["id"]}
for author_id, abstract in author_publication_pairs:
    if abstract and isinstance(abstract, str) and re.match('^(?=.*[a-zA-Z])', abstract):
        language = detect(abstract)
        if language == 'en':
            authors_texts[author_id].append(abstract)

In [153]:
texts = authors_texts[829]

remove_pos = ['ADV', 'PRON', 'PART', 'DET', 'SPACE', 'NUM', 'SYM', 'ADP', 'VERB', 'CCONJ']
stop_words = ['paper', 'present', 'propose', 'datum', 'people', 'result', 'solution', 'case', 'order', 'base', 'ieee', 'privacy', 'policy',
             'new', 'old', 'context', 'high', 'different', 'new', 'old', 'research', 'type', 'approach', 'important', 'main', 'range',
             'helpful', 'large', 'difficult', 'available', 'amount', 'useful', 'importance', 'article', 'abstract', 'scale', 'copyright',
             'real', 'quality', 'inconvenience', 'benefit', 'unavailable', 'term', 'condition', 'interest', 'organization', 'use',
             'task', 'student', 'professor', 'teacher', 'university', 'conference', 'thank']

# keep only adjectives and nouns
remove_entities = ['PERSON', 'NORP', 'FAC', 'GPE', 'LOC', 'LANGUAGE', 'DATE', 'TIME', 'PERCENT', 'MONEY',
                   'QUANTITY', 'CARDINAL', 'ORDINAL']

tokens = []
for abstract in texts:
    doc = nlp(abstract)
    t = [token.lemma_.lower() for token in doc if token.is_alpha and token.ent_type_ not in remove_entities
                             and token.lemma_.lower() not in stop_words and token.pos_ not in remove_pos and not token.is_stop]
    tokens.append(t)

# add bigrams to the token list
bigram = Phrases(tokens, min_count=2, delimiter=' ', threshold=1)
tokens = [bigram[text] for text in tokens]
trigram = Phrases(tokens, min_count=2, delimiter=' ', threshold=1)
tokens = [trigram[text] for text in tokens]

In [154]:
# remove single words
tokens = [[token for token in text if len(token.split(" ")) > 1] for text in tokens]

In [155]:
# create dictionary with gensim
dictionary = Dictionary(tokens)
# create corpus
corpus = [dictionary.doc2bow(text) for text in tokens]

In [158]:
# apply LDA
lda_model = LdaMulticore(corpus=corpus, id2word=dictionary, iterations=300, num_topics=1, workers=4, passes=50)
topics = lda_model.print_topics(-1)

for idx, topic in topics:
    print(topic)

0.013*"sar image" + 0.008*"synthetic aperture radar sar" + 0.008*"satellite image" + 0.005*"earth observation" + 0.005*"feature extraction" + 0.005*"image content" + 0.004*"terrasar x" + 0.004*"image patch" + 0.003*"land cover" + 0.003*"remote image"


EXTRACT KEYWORDS FROM PUBLICATION TITLES

In [84]:
# make a dictionary containing the combined titles for each author
author_title_pairs = list(zip(publications['user_id'], publications['title']))

authors_titles = {author_id:"" for author_id in authors["id"]}
for author_id, title in author_title_pairs:
    if title and isinstance(title, str):
        authors_titles[author_id] += "\n" + title

In [86]:
kw_extractor = yake.KeywordExtractor()
max_ngram = 2
deduplication_threshold = 0.2
keywords_nr = 3
custom_kw_extractor = yake.KeywordExtractor(lan="en", n=max_ngram, dedupLim=deduplication_threshold, top=keywords_nr, features=None)

text = authors_titles[562]
doc = nlp(text)

transformed_text = ' '.join([token.text for token in doc])

keywords = custom_kw_extractor.extract_keywords(transformed_text)
for kw in keywords:
    print(kw)

('Cloud Computing', 3.0745344837092404e-06)
('Big Data', 3.2118868621537734e-06)
('distributed systems', 3.458271355126227e-06)
